In [ ]:
!pip install streamlit
!pip install joblib  # For Pickle
import streamlit as st
import joblib

In [ ]:
joblib.dump(kmeans, 'kmeans_model.pkl')
kmeans = joblib.load('kmeans_model.pkl')

In [ ]:
%%writefile music_app.py
import streamlit as st
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
df=pd.read_csv("data.csv")
df["duration"]=df["duration_ms"].apply(lambda x: round(x/1000))
df.drop("duration_ms", inplace=True, axis=1)
# Feature selection
features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'loudness', 'speechiness', 'tempo', 'duration']

# Create a new DataFrame with only the selected features
song_features = df[features]

# Normalize the features (optional, but recommended)
song_features_normalized = (song_features - song_features.mean()) / (song_features.max() - song_features.min())
total_dr = df['duration']  # Assuming 'duration' is a column in your DataFrame
years = df['year']
kmeans = KMeans(n_clusters=10, random_state=42)

kmeans.fit(song_features_normalized)

# Streamlit app
st.title("Song Recommendation System")

# Input for song name
song_name = st.text_input("Enter a song name:")

# Button to trigger recommendations
if st.button("Get Recommendations"):
    # Function to get recommendations
    def get_recommendations(song_name, kmeans_model=kmeans, data=df):
        song_index = df[df['name'] == song_name].index[0]
        cluster_label = kmeans_model.labels_[song_index]
        cluster_songs = df[kmeans_model.labels_ == cluster_label]['name']
        return cluster_songs

    # Get recommendations for the input song
    recommended_songs = get_recommendations(song_name)

    # Display recommendations
    if len(recommended_songs) > 0:
        st.subheader("Recommended Songs:")
        st.write(recommended_songs)
    else:
        st.write("No recommendations found for this song.")

In [ ]:
!pip install pyngrok
from pyngrok import ngrok
!streamlit run music_app.py & npx localtunnel --port 8501